In [1]:
import os
import numpy as np
import pandas as pd

from mpire import WorkerPool

RAW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
RAW_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
NEW_SNP_PATH = "/data1/zhoujb/cowpeaSequence/snpDir/"
NEW_RES_PATH = "/data1/zhoujb/cowpeaSequence/Result/"
REF_PATH = "/data1/zhoujb/cowpeaSequence/refernce/"
TEMP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/templateSNP/"

In [3]:
def getIdentity(dir_name):
    
    tmep_dir = os.path.join(TEMP_PATH, dir_name)
    tmep_df = pd.DataFrame()
    for tmep_file in os.listdir(tmep_dir):
        tmp_file = pd.read_table(os.path.join(tmep_dir, tmep_file), index_col="Index")
        tmep_df = pd.concat([tmep_df, tmp_file], axis=1, sort=False)

    four_raw = pd.read_table(os.path.join(NEW_SNP_PATH, "{}.txt".format(dir_name)), index_col="Pos")
    four_raw = four_raw.drop(columns=['Chr'])

    if len(four_raw) != len(tmep_df):
        raise ValueError("{}".format(dir_name))

    tmep_df = tmep_df.loc[four_raw.index]
    out_df = pd.DataFrame(columns=four_raw.columns, index=tmep_df.columns)
    for col in out_df.columns:
        for index in out_df.index:
            comb_df = pd.concat([tmep_df[index], four_raw[col]], axis=1, sort=False)
            identity = (comb_df[index] == comb_df[col]).sum() / comb_df.shape[0]
            out_df.loc[index, col] = identity

    out_df.to_csv(os.path.join(NEW_SNP_PATH, "{}.identity".format(dir_name)), sep="\t")
    return

if __name__ == "__main__":
    temp_dir = os.listdir(TEMP_PATH)

    with WorkerPool(n_jobs=48) as pool:
        pool.map(getIdentity, temp_dir, progress_bar=True)

    print("ALL DONE!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 805/805 [00:01<00:00, 451.17it/s]

ALL DONE!


In [2]:
raw_gen_df = pd.read_table(os.path.join(RAW_RES_PATH, "Genotype_by_Hap_ID.tsv"), 
                           usecols=['Region_index', 'Chr', 'Start', 'End'], index_col='Region_index')
geno_hap_four = raw_gen_df.copy()
for index in raw_gen_df.index:
    tmp_file = pd.read_table(os.path.join(NEW_SNP_PATH, "{}.identity".format(index)), index_col=0)
    tmp_hap = tmp_file.idxmax(axis=0)
    
    for tmp_index in tmp_hap.index:
        if tmp_file[tmp_index].max() < 0.5:
            tmp_hap.loc[tmp_index] = "-"
    geno_hap_four.loc[index, tmp_hap.index] = tmp_hap.values

geno_hap_four.to_csv(os.path.join(NEW_RES_PATH, "Genotype_by_Hap_four.tsv"), sep="\t")
print("ALL DONE!")

ALL DONE!


In [3]:
geno_hap_four

,Chr,Start,End,LP001,LP012,LP024,zhijiang618
Region_index,,,,,,,
Chr01_51400001_51530000,Chr01,51400001,51530000,Chr01_51400001_51530000_218_IH_D367,Chr01_51400001_51530000_218_IH_D367,Chr01_51400001_51530000_218_IH_D367,Chr01_51400001_51530000_218_IH_D367
Chr01_4550001_4690000,Chr01,4550001,4690000,Chr01_4550001_4690000_209_IH_D369,Chr01_4550001_4690000_89_VH_D366,Chr01_4550001_4690000_209_IH_D369,Chr01_4550001_4690000_209_IH_D369
Chr01_44170001_44300000,Chr01,44170001,44300000,Chr01_44170001_44300000_291_PH_D365,Chr01_44170001_44300000_291_PH_D365,Chr01_44170001_44300000_291_PH_D365,Chr01_44170001_44300000_291_PH_D365
Chr01_4080001_4260000,Chr01,4080001,4260000,Chr01_4080001_4260000_178_IH_D612,Chr01_4080001_4260000_178_IH_D612,Chr01_4080001_4260000_178_IH_D612,Chr01_4080001_4260000_178_IH_D612
Chr01_5960001_6170000,Chr01,5960001,6170000,Chr01_5960001_6170000_260_PH_D435,Chr01_5960001_6170000_9_NH_D634,Chr01_5960001_6170000_260_PH_D435,Chr01_5960001_6170000_260_PH_D435
...,...,...,...,...,...,...,...
Chr09_19940001_20070000,Chr09,19940001,20070000,Chr09_19940001_20070000_17_NH_D388,Chr09_19940001_20070000_3_NH_D409,Chr09_19940001_20070000_261_PH_D365,Chr09_19940001_20070000_261_PH_D365
Chr09_19770001_19920000,Chr09,19770001,19920000,Chr09_19770001_19920000_237_IH_D367,Chr09_19770001_19920000_237_IH_D367,Chr09_19770001_19920000_237_IH_D367,Chr09_19770001_19920000_237_IH_D367
Chr09_41170001_41280000,Chr09,41170001,41280000,Chr09_41170001_41280000_16_NH_D376,Chr09_41170001_41280000_258_PH_D365,Chr09_41170001_41280000_258_PH_D365,Chr09_41170001_41280000_258_PH_D365
